# Bayes by Backprop: Mushroom Contextual Bandit

__Background - Multi-armed and Contextual Bandit problems:__ <br>
_“Bandit”_ in _“multi-armed bandits”_ comes from _“one-armed bandit”_ machines used in a casino. Imagine that you are in a casino with many one-armed bandit machines. Each machine has a different probability of a win. Your goal is to maximize total payout. You can pull a limited number of arms, and you don’t know which bandit to use to get the best payout. The problem involves exploration/exploitation tradeoff: you should balance between trying different bandits to learn more about an expected payout of every machine, but you also want to exploit the best bandit you know about more. While multi-armed bandits select the strategy which maximises the expected reward without taking into consideration the state of the environment, _contextual bandits_ output an action based on the context.

__Mushroom Case Stuty:__ <br>
We are provided with a list of 8124 mushrooms, each having 22 features (characteristcs of the mushroom) and 1 label (poisonous or edible). Our agent can carry out 2 actions: eat a mushroom or not eat a mushroom. The problem context is the vector of features which is associated with the mushroom which the agent is about to eat/non eat.
If our agent eats an edible mushroom, it receives a reward of 5. If the agent eats a poisonous mushroom, it receives a reward of -35 with 0.5 probability and a reward of 5 with 0.5 probaility. If the agent doesn't eat, it receives a reward o 0.

We are also provided an _oracle_. The oracle always selects the right action, receiving a reward of 5 when it eats an edible mushroom, and a reward of 0 when it doesn't eat.

__Objective:__ <br>
Create a BNN which minimises the _cumulative regret_ of the agent. Regreat measures the difference between the oracle and our agent's reward.

__BNN Architecture:__
- 2 hidden layers
- Each layer has 100 rectified units
- Vector input: vector consisting of the mushroom features (context) and a one of _K_ encoding of the action
- Output: single scalar representing the expected reward of the given action in the given context

__Additional Important Information:__
- To calculate the expected reward for an action, Google samples twice the weights and averages the outputs
- To train the network, Google keeps a buffer of 4096 mushrooms.
- For training, Google randomly draws minibatches of size 64 for 64 training steps.
- After every training stage, the agent interacts with a new mushroom

## TASKS:
- Data preparation: DONE
- Reward function: DONE
- Initialise buffer function: DONE
- Update buffer function: DONE
- Create class for Gaussian distribution: DONE
- Create class for generating prior distribution: DONE
- Create class for generating BNN layer: DONE
- Create BNN: Need to change
- Create function for creating network estimates: TO BE DONE
- Create function for updating network parameters: TO BE DONE


### Algorithm for network estimtes:
##### Inputs: network and mushroom - Returns rewards for both actions
def network-estimates(network, mushroom): <br>
    sample w1,w2<br>
    r1 = 0.5 * (f(mushroom,eat, w1) + f(mushroom,don't eat, w1))<br>
    r2 = 0.5 * (f(mushroom,eat, w2) + f(mushroom,don't eat, w2))<br>
    return r1,r2<br>
    


In [210]:
# Import Libraries

%matplotlib inline
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import preprocessing
from sklearn import metrics
import random

## __Data Preparation:__ ## 
Input is the data file. Output is an array of one-hot encoded labels (y, shape: 8124) and an array of context features (x, shape: 8124 elements, each with 117 features). 117 features derive from doing a one-hot encoding on the feature classes.

In [211]:
# Import data from file
df = pd.read_csv(os.getcwd() + '/agaricus-lepiota.data', sep=',', header=None,
             error_bad_lines=False, warn_bad_lines=True, low_memory=False)

# Set pandas to output all of the columns in output
df.columns = ['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment',
         'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root',
         'stalk-surf-above-ring','stalk-surf-below-ring','stalk-color-above-ring','stalk-color-below-ring',
         'veil-type','veil-color','ring-number','ring-type','spore-color','population','habitat']

# Split context from label
X = pd.DataFrame(df, columns=df.columns[1:len(df.columns)], index=df.index)
# Put the class values (0th column) into Y
Y = df['class']

# Transform labels into one-hot encoded array
le = preprocessing.LabelEncoder()
le.fit(Y)
y = le.transform(Y)

# Temporary variable to avoid error 
x_tmp = pd.DataFrame(X,columns=[X.columns[0]])

# Encode each feature column and add it to x_train 
for colname in X.columns:
    le.fit(X[colname])
    #print(colname, le.classes_)
    x_tmp[colname] = le.transform(X[colname])

# Produce mushroom array: 8124 mushrooms, each with 117 one-hot encoded features
oh = preprocessing.OneHotEncoder(categorical_features='all')
oh.fit(x_tmp)
x = oh.transform(x_tmp).toarray()

/Users/lorcandelaney/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/lorcandelaney/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:380: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.
  DeprecationWarning)


## __Reward function:__ ##
Implemented as described above

In [212]:
# REWARD FUNCTION

def reward_function(action_chosen, label):
    # REWARDS FOR AGENT
    #  Eat poisonous mushroom
    if action_chosen == 0 and label == 1:
        if np.random.rand() > 0.5:
            reward = -35
        else:
            reward = 5
    #  Eat edible mushroom
    elif action_chosen == 0 and label == 0:
        reward = 5
    # Do not eat nay mushroom
    else:
        reward = 0
    
    # REWARD FOR ORACLE
    if label == 1:
        oracle_reward = 0
    else:
        oracle_reward = 5
    return reward, oracle_reward

## Create Mushroom Buffer
Generate buffer to keep 4096 elements used to train the Bayesian Neural Network

In [213]:
# Initialise Buffer: an array of arrays. Each array contains the context of a mushroom, a randomly 
# selected action for the mushroom and the respective reward obtained when taking the action


def init_buffer():

    init_buffer = []

    for i in range(4096):
        
        # Generate random action
        init_action = random.randint(0,1)

        # Calculate the expected reward by calling the reward_function. The inputs of the reward function are 
        # the action and the label of the mushroom (edible/poisonous). Returns expected reward for agent and for the oracle
        init_rewards = reward_function(init_action, y[i])

        # We take only the reward for the action since we include only this in the buffer
        init_agent_rewards = init_rewards[0]

        # Create one element of the buffer and append it to the buffer array. Each element of the buffer is an array 
        # which includes the expected reward, the action taken and the mushroom context
        buffer_element = [init_agent_rewards, init_action]
        element = np.concatenate((buffer_element, x[i]))
        buffer.append(element)
    
    return init_buffer



In [214]:
# Define method for uopdating the buffer after each training stage.
# Inputs: current state of the buffer, the action taken on the previous mushroom, the obtained reward and the context
# Outputs: pop the last element in the buffer, append the element formed with the inputs and return the updated buffer

def buffer_update(buffer, mushroom, action, reward, context):
    
    element = [reward, action]
    new_buffer_element = np.concatenate((element, context))
    buffer.pop()
    buffer.append(new_element)

    return buffer


## Neural Network


In [215]:
# Define some hyperparameters
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
print("Cuda available?: ",torch.cuda.is_available())

PI = 0.5
SIGMA_1 = torch.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.FloatTensor([math.exp(-6)])

Cuda available?:  False


### Create class for Gaussian distribution

In [216]:
### Taken from source

class Gaussian(object):
    def __init__(self, mu, rho):
        super().__init__()
        self.mu = mu
        self.rho = rho
        self.normal = torch.distributions.Normal(0,1)
    
    @property
    def sigma(self):
        return torch.log1p(torch.exp(self.rho))
    
    def sample(self):
        epsilon = self.normal.sample(self.rho.size()).to(DEVICE)
        return self.mu + self.sigma * epsilon
    
    def log_prob(self, input):
        return (-math.log(math.sqrt(2 * math.pi))
                - torch.log(self.sigma)
                - ((input - self.mu) ** 2) / (2 * self.sigma ** 2)).sum()

### Create class for generating mixed gaussian prior distribution

In [217]:
### Taken from source

class ScaleMixtureGaussian(object):
    def __init__(self, pi, sigma1, sigma2):
        super().__init__()
        self.pi = pi
        self.sigma1 = sigma1
        self.sigma2 = sigma2
        self.gaussian1 = torch.distributions.Normal(0,sigma1)
        self.gaussian2 = torch.distributions.Normal(0,sigma2)
    
    def log_prob(self, input):
        prob1 = torch.exp(self.gaussian1.log_prob(input))
        prob2 = torch.exp(self.gaussian2.log_prob(input))
        return (torch.log(self.pi * prob1 + (1-self.pi) * prob2)).sum()
    
    

### Create class for generating BNN Layer

In [218]:
### Taken from source

class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        # Weight parameters
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-0.2, 0.2))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-5,-4))
        self.weight = Gaussian(self.weight_mu, self.weight_rho)
        # Bias parameters
        self.bias_mu = nn.Parameter(torch.Tensor(out_features).uniform_(-0.2, 0.2))
        self.bias_rho = nn.Parameter(torch.Tensor(out_features).uniform_(-5,-4))
        self.bias = Gaussian(self.bias_mu, self.bias_rho)
        # Prior distributions
        self.weight_prior = ScaleMixtureGaussian(PI, SIGMA_1, SIGMA_2)
        self.bias_prior = ScaleMixtureGaussian(PI, SIGMA_1, SIGMA_2)
        self.log_prior = 0
        self.log_variational_posterior = 0

    def forward(self, input, sample=False, calculate_log_probs=False):
        if self.training or sample:
            weight = self.weight.sample()
            bias = self.bias.sample()
        else:
            weight = self.weight.mu
            bias = self.bias.mu
        if self.training or calculate_log_probs:
            self.log_prior = self.weight_prior.log_prob(weight) + self.bias_prior.log_prob(bias)
            self.log_variational_posterior = self.weight.log_prob(weight) + self.bias.log_prob(bias)
        else:
            self.log_prior, self.log_variational_posterior = 0, 0

        return F.linear(input, weight, bias)

### Generate BNN architecture

In [220]:
### To be changed

class BayesianNetwork(nn.Module):
    def __init__(self, inputSize, CLASSES, layers, activations, SAMPLES, BATCH_SIZE, NUM_BATCHES):
        super().__init__()
        self.inputSize = inputSize
        self.activations = activations
        self.CLASSES = CLASSES
        self.SAMPLES = SAMPLES
        self.BATCH_SIZE = BATCH_SIZE
        self.NUM_BATCHES = NUM_BATCHES
        self.DEPTH = 0

        self.layers = nn.ModuleList([])
        if layers.size == 0:
            self.layers.append(BayesianLinear(inputSize, CLASSES))
            self.DEPTH += 1
        else:
            self.layers.append(BayesianLinear(inputSize, layers[0]))
            self.DEPTH += 1
            for i in range(layers.size-1):
                self.layers.append(BayesianLinear(layers[i], layers[i+1]))
                self.DEPTH += 1
            self.layers.append(BayesianLinear(layers[layers.size-1], CLASSES))
            self.DEPTH += 1
            
    def forward(self, x, sample=False):
        x = x.view(-1, self.inputSize)
        layerNumber = 0
        for i in range(self.activations.size):
            if self.activations[i]=='relu':
                x = F.relu(self.layers[layerNumber](x, sample))
            elif self.activations[i]=='softmax':
                x = F.log_softmax(self.layers[layerNumber](x, sample), dim=1)
            else:
                x = self.layers[layerNumber](x, sample)
            layerNumber+= 1
        return x
    
    def log_prior(self):
        value = 0
        for i in range(self.DEPTH):
            value+= self.layers[i].log_prior
        return value
    
    def log_variational_posterior(self):
        value = 0
        for i in range(self.DEPTH):
            value+= self.layers[i].log_variational_posterior
        return value
    
    def sample_elbo(self, input, target):
        samples=self.SAMPLES
        outputs = torch.zeros(samples, self.BATCH_SIZE, self.CLASSES).to(DEVICE)
        log_priors = torch.zeros(samples).to(DEVICE)
        log_variational_posteriors = torch.zeros(samples).to(DEVICE)
        negative_log_likelihood = torch.zeros(samples).to(DEVICE)
        
        for i in range(samples):
            outputs[i] = self.forward(input, sample=True)
            log_priors[i] = self.log_prior()
            log_variational_posteriors[i] = self.log_variational_posterior()
            if self.CLASSES == 1:
                negative_log_likelihood[i] = (.5 * (target - outputs[i]) ** 2).sum()
            
        log_prior = log_priors.mean()
        log_variational_posterior = log_variational_posteriors.mean()
        if self.CLASSES > 1:
            negative_log_likelihood = F.nll_loss(outputs.mean(0), target, size_average=False)
        else:
            negative_log_likelihood = negative_log_likelihood.mean()
        loss = (log_variational_posterior - log_prior)/self.NUM_BATCHES + negative_log_likelihood
        return loss, log_prior, log_variational_posterior, negative_log_likelihood